In [10]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

import numpy as np
import pandas as pd 
import json

import matplotlib.pyplot as plt
from transformers import BertModel
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import BertTokenizer, get_linear_schedule_with_warmup

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import matthews_corrcoef, confusion_matrix

import random
import wandb
#%pip install deepl
import deepl

import sys
sys.path.append('/home/iñaki/host_data')
from models import BertBasePooledOutput
from CreateDataset import BertDataset, createAuxDeepLDataset

In [11]:
########################################
#                Ideas                 #
########################################

# Data Augmentation
    # Crear x10 el numero de elementos del dataset mediante LLMs
    # Una idea es utilizar como propmpt dos o tres textos aleatorios originales etiquetados igual y probar a generar un nuevo texto.
        # Se podria aletorizar tambien el propmt para variar los estilos de escritura. 
        #        (Persona de nivel cultural bajo, Entendido, Pretencioso, Misterioso, paranoico, asetivo, profesor) 
        #     Adjetivos que definen el estilo  
        # Intrincado, Descriptivo, Cautivador, Persuasivo, Elegante, Intenso, Imaginativo, Sobrio, Oscuro, Profundo, Emotivo, 
        # Surrealista, Irreverente, Oscilante, Contundente, Desgarrador, Espeluznante, Enigmático, Inquietante, Mordaz"""
        # Se podría entrenar inicialmente con estos datos, y finalmente con los originales.  
        # Finalmente testear con test, que no ha sido usado para generar prompts
        
    # Otros metodos  -  https://neptune.ai/blog/data-augmentation-nlp
        # Back translation
        # Easy methods
        #     Synonym Replacement  |  Random synonym Insertion   |   Random Swap   |    Random Deletion
        # Albumentation (libreria)
        #     Shuffle Sentences Transform 
        # Libreria para muchos de estos métodos  -  NLPAug 
        
    # Aumentado sobre embedding
        # Mezclar 2 embeddings
        # Añadir ruido gaussiano
        # Ruido adversial
        # MixUp  (mezclar embedding y su label en la proporcion dada por una funcion)
        
    # Consistency regularization ??
    
    # Contrastive learning ??

# TRADUCIR EL DATASET ESPAÑOL A INGLES Y PROBAR

# DUDAS
    # Guardo la mejor epoch?

# TO DO tutorias
    # Guardar logits baseline para probar 
    # Guardar modelos para probar forma de ensembles
    # repetir enseble 10 veces (baja sigma a la mitad)
    # Tecnicas basicas de data aumentacion
    # Baseline 10 folds sobre test, para tener 10 test como al probar 10 ensembles
# ----------
     
    # Analizar matriz de confusion sobre baseline
    
    # Guardar las predicciones de los k-modelos
    
    # Bert ajustado para mensajes de redes sociales (telegram)
    
# Recently DONE
    # Plotear distribucion de errores
    # implementar Predicciones 0, 1 o no lo se
    # Matriz de confusion
    # probar los k modelos sobre test
    # Guardar ultimo epoch de validacion, quitar MCC cada epoch, gaurdar media y desviacion del ultimo
    
    # Mirar si tiene emojis y traducirlos a texto (emojify)
    
# DONE
    #  ¿¿¿ #os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8" ???
       # ERROR  # torch.use_deterministic_algorithms(True)  
    # guardar los resultados weight and bias o ML flow
    # Intentar guardar el MCC de cada fold para poder sacar desviaciones
    # Guardar semilas.  La semilla es para que los datos sean los mismos.
    # MCC  poner etiquetas a 1 y 2  ¿¿¿ seguro ???
    # Arreglar docker
    # sacar 10% de datos del dataset como sanity-check 
    # split stratify   (sklearn viene)
    # cross validation (sklearn)

In [12]:
###############################
#       configuración         #
###############################
MODEL_NAME = "BertBasePooledOutput"

MAX_LENGTH = 512
HEAD_DROPOUT = 0
DATA_AUGMENTATION = ["Oversampling"]
FOLDS_NUM = 10

NUM_EPOCHS = 5
LEARNING_RATE = 2e-5
BATCH_SIZE = 16

FREEZE_BACKBONE = True
FREEZE_BACKBONE_EPOCHS = 7


datadir = "/home/iñaki/host_data/dataset_oppositional/"

In [13]:
# Cargamos los datasets de entrenamiento y test
train_en_dataset_path = datadir + "train_en_data.pth"
test_en_dataset_path = datadir + "test_en_data.pth"
train_es_dataset_path = datadir + "train_es_data.pth"
test_es_dataset_path = datadir + "test_es_data.pth"

# creamos los datasets de test
X, y = torch.load(test_es_dataset_path)
test_es_dataset = BertDataset(X, y)
X, y = torch.load(test_en_dataset_path)
test_en_dataset = BertDataset(X, y)

#Creamos dataloader de test
test_en_dataloader = DataLoader(test_en_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [14]:
def train_loop(model, train_dataloader, optimizer, scheduler, NUM_EPOCHS):
    train_losses = []
    for epoch in range(NUM_EPOCHS):
        # Set your model to training mode
        total_loss = 0
        total_train_samples = 0
        model.train()
        with tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", unit="batch") as tepoch:
            for i, data in enumerate(tepoch):
                input_ids, attention_mask, labels, _ = data
                # Move batch to device
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                # Forward pass
                optimizer.zero_grad()
                outputs = model(input_ids, attention_mask)
                logits = outputs.squeeze(-1)

                # Compute loss
                loss = criterion(logits, labels.float())

                # Backward pass
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

                # Update tqdm description with loss
                tepoch.set_postfix(loss=loss.item())
                
                #update learning rate
                if scheduler:
                    scheduler.step()

            # Print average loss for this epoch
            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Epoch {epoch+1}, Average Train Loss: {avg_train_loss}")
            train_losses += [avg_train_loss]
            
    return train_losses

In [15]:
def evaluate(model, test_dataloader, THRESHOLD=0.5, LOWER_UPPER_BOUND=False, val_or_test=None):
    test_outputs = []
    test_true_labels = []
    with torch.no_grad():
        total_test_loss = 0
        total_test_samples = 0
        for data in test_dataloader:
            input_ids, attention_mask, labels, _ = data
            # Save list of ground truth labels for metrics
            test_true_labels += labels.tolist()
                    
            # Move batch to device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
                    
            # Forward pass
            outputs = model(input_ids, attention_mask)
            test_outputs += outputs.tolist()
            logits = outputs.squeeze(-1)

            # Compute loss
            test_loss = criterion(logits, labels.float())

            # Accumulate test loss and total number of samples
            total_test_loss += test_loss.item() * labels.size(0)
            total_test_samples += labels.size(0)
            
    # Calculate average test loss
    average_test_loss = total_test_loss / total_test_samples   
                    
    test_predictions = [1 if x[0] > THRESHOLD else 0 for x in test_outputs]
    mcc = matthews_corrcoef(test_true_labels, test_predictions)
    print(f"{val_or_test} MCC {THRESHOLD} threshold:  {mcc:.4f}")
    
    # log in wandb
    if wandb.run is not None:
        wandb.log({f"{val_or_test}_{THRESHOLD}_threshold_MCC":mcc})

    ##### matriz de confusion
    cm = confusion_matrix(test_true_labels, test_predictions)
    print(cm)

    #### get index of errors in predictions
    errors = [i for i, x in enumerate(test_predictions) if x != test_true_labels[i]]
    #### get the model outputs for the errors
    errors_outputs = [test_outputs[i] for i in errors]
    #### plot the errors with title
    plt.hist([x[0] for x in errors_outputs]) 
    plt.title(f"errors distribution")
    plt.show()

    
    # Si hay valores para el rango de valor desconocido sobre-escribimos las predicciones  
    if LOWER_UPPER_BOUND != False:
        test_predictions = [1 if x[0] > LOWER_UPPER_BOUND[1] else (0 if x[0] <= LOWER_UPPER_BOUND[0] else -1) for x in test_outputs]
        mcc = matthews_corrcoef(test_true_labels, test_predictions)
        print(f"{val_or_test} MCC with Lower_bound:{LOWER_UPPER_BOUND[0]} and upper_bound:{LOWER_UPPER_BOUND[1]} > MCC: {mcc:.4f}")
        # log in wandb
        if wandb.run is not None:
            wandb.log({f"{val_or_test}_lower_upper_{LOWER_UPPER_BOUND[0]}-{LOWER_UPPER_BOUND[1]}_MCC":mcc})
            
    # Print average test loss for this epoch
    print(f"Average {val_or_test} Loss: {average_test_loss:.4f}")
    
    return test_outputs

In [16]:
def evaluate_kfold_ensemble(predictions, test_dataloader, THRESHOLD=0.5, LOWER_UPPER_BOUND=False):
    test_true_labels = []
    with torch.no_grad():
        for data in test_dataloader:
            input_ids, attention_mask, labels, _ = data
            # Save list of ground truth labels for metrics
            test_true_labels += labels.tolist()
            
    # Mean output
    test_outputs = np.array(predictions)
    print(test_outputs.shape)
    test_outputs = test_outputs.mean(axis=0)
    print(test_outputs.shape)
    test_mean_predictions = [1 if x > THRESHOLD else 0 for x in test_outputs]
    mcc = matthews_corrcoef(test_true_labels, test_mean_predictions)
    print(f"test ensemble MCC {THRESHOLD} threshold: {mcc:.4f}")
    # log in wandb
    if wandb.run is not None:
        wandb.log({f"test_ensemble_{THRESHOLD}_threshold_MCC":mcc})
    # Max Voting

In [8]:
##################################
#     Weigths and bias Login    #
##################################
# MY API KEY
#    240062b42b3367d962eecb38802f856b41245735
#wandb.login(key="240062b42b3367d962eecb38802f856b41245735")


for i in range(1):
    print(f"experimento {i}:")
    #####################
    #  Reproducibilidad #
    #####################
    SEED = random.randint(0,10000)
    print(f"seed: {SEED}")
    # fuente  -  https://pytorch.org/docs/stable/notes/randomness.html
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.deterministic = True


    ######################################
    #          K-FOLD TRAIN LOOP         #
    ######################################

    # CROSS VALIDATION LOOP
    X, y = torch.load(train_en_dataset_path)
    X = np.array(X)
    y = np.array(y)

    skf = StratifiedKFold(n_splits=FOLDS_NUM, shuffle=True, random_state=42)

    k_test_outputs = []
    # Loop over the folds
    for fold_idx, (train_index, val_index) in enumerate(skf.split(X, y)):
        print(f"fold {fold_idx + 1}")
        
        # Weigths and bias init
        config = {
        "MAX_LENGTH": MAX_LENGTH,
        "HEAD_DROPOUT": HEAD_DROPOUT,
        "DATA_AUGMENTATION": DATA_AUGMENTATION,
        "FOLDS_NUM": FOLDS_NUM,
        "NUM_EPOCHS": NUM_EPOCHS,
        "LEARNING_RATE": LEARNING_RATE,
        "BATCH_SIZE": BATCH_SIZE,
        "FOLD_IDX": fold_idx,
        "SEED": SEED
        }
        GROUP = f"{HEAD_DROPOUT}_{BATCH_SIZE}_{LEARNING_RATE}"
        wandb.init(project="BASELINE", group=f"Group_{i}", config=config)
        
        # Create the fold-specific train and validation sets
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]
        
        train_en_dataset = BertDataset(X_train_fold, y_train_fold)
        val_en_dataset = BertDataset(X_val_fold, y_val_fold)
        
        # Create the train dataloader with a random oversampling of the minority class
        if "Oversampling" in DATA_AUGMENTATION:
            # Oversampling Train Dataloader
            labels = [sample[2] for sample in train_en_dataset]
            class_counts = torch.bincount(torch.tensor(labels))
            class_weights = 1.0 / class_counts.float()

            weights = [class_weights[label] for label in labels]
            sampler = WeightedRandomSampler(weights=weights, num_samples=len(train_en_dataset), replacement=True)
            train_en_dataloader = DataLoader(train_en_dataset, sampler=sampler, batch_size=BATCH_SIZE)
            print(f"Oversampling")
        else:
            print(f"No Oversampling")
            train_en_dataloader = DataLoader(train_en_dataset, batch_size=BATCH_SIZE, shuffle=True)
            
        val_en_dataloader = DataLoader(val_en_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        # Instanciamos el modelo
        model = BertBasePooledOutput(dropout_prob=HEAD_DROPOUT)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(device)
        model.to(device)
        
        # Paralelizamos el modelo si hay mas de una GPU
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model, device_ids=[0, 1])
            
        # Define your loss function (binary cross-entropy in this case)
        criterion = nn.BCELoss()
          
        # Activamos todas las capas
        for param in model.parameters():
            param.requires_grad = True
        
        # Contmaos los parametros entrenables
        print(f"total params: {sum(p.numel() for p in model.parameters())} trainble params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
        

        # Define your optimizer
        optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
        
        # Define your learning rate scheduler
        total_steps = len(train_en_dataloader) * NUM_EPOCHS
        warmup_steps = int(total_steps * 0.1)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

        train_losses = train_loop(model, train_en_dataloader, optimizer, scheduler, NUM_EPOCHS)   
        
        _ = evaluate(model, val_en_dataloader, THRESHOLD=0.5, val_or_test="Val")
        k_test_outputs += [evaluate(model, test_en_dataloader, THRESHOLD=0.5, LOWER_UPPER_BOUND=[0.4, 0.6], val_or_test="Test")]
        
        # Save the model
        torch.save(model.state_dict(), f"/home/iñaki/host_data/checkpoints/Baseline/{MODEL_NAME}_fold_{fold_idx}.pth")
    evaluate_kfold_ensemble(k_test_outputs, test_en_dataloader)
        
    # bB - Finalizamos el experimento
    wandb.finish()

experimento 0:
seed: 2635


fold 1


wandb: Currently logged in as: inakitodc (inaki). Use `wandb login --relogin` to force relogin


cuda
total params: 109483009 trainble params: 109483009


Epoch 1/5:   0%|          | 1/203 [00:05<18:46,  5.58s/batch, loss=0.665]


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/iñaki/host_data/models.py", line 15, in forward
    outputs = self.bert(input_ids, attention_mask=attention_mask)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py", line 1022, in forward
    encoder_outputs = self.encoder(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dis

In [ ]:
#### Bucle cargando todos los modelos BASELINE
baseline_test_outputs = []

for model_path in os.listdir("/home/iñaki/host_data/checkpoints/Baseline/"):
    model = BertBasePooledOutput(dropout_prob=HEAD_DROPOUT)
    model.load_state_dict(torch.load(f"/home/iñaki/host_data/checkpoints/Baseline/{model_path}"))
    model.to(device)
    model.eval()
    baseline_test_outputs += [evaluate(model, test_en_dataloader, THRESHOLD=0.5, val_or_test="Test")]


# Evaluamos el valor optimo para el threshold

test_true_labels = []
for data in test_en_dataloader:
    input_ids, attention_mask, labels, _ = data
    test_true_labels += labels.tolist()

test_true_labels

all_mcc = []
threshold = []
for i in range(101):
    test_outputs = np.array(baseline_test_outputs)
    print(test_outputs.shape)
    test_outputs = test_outputs.mean(axis=0)
    print(test_outputs.shape)
    test_mean_predictions = [1 if x > i/100 else 0 for x in test_outputs]
    mcc = matthews_corrcoef(test_true_labels, test_mean_predictions)
    all_mcc += [mcc]
    threshold += [i/100]

# plot a mcc vs threshold
plt.plot(all_mcc, threshold)
plt.show()

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)